## --- Day 19: Monster Messages ---
    
You land in an airport surrounded by dense forest. As you walk to your high-speed train, the Elves at the Mythical Information Bureau contact you again. They think their satellite has collected an image of a sea monster! Unfortunately, the connection to the satellite is having problems, and many of the messages sent back from the satellite have been corrupted.

They sent you a list of the rules valid messages should obey and a list of received messages they've collected so far (your puzzle input).

The rules for valid messages (the top part of your puzzle input) are numbered and build upon each other.
* Some rules, like **3: "b"**, simply match a single character (in this case, b).
* The remaining rules **list the sub-rules that must be followed**; for example, the rule 0: 1 2 means that to match rule 0, the text being checked must match rule 1, and the text after the part that matched rule 1 must then match rule 2.
* Some of the rules have **multiple lists of sub-rules separated by a pipe (|)**. This means that at least one list of sub-rules must match. (The ones that match might be different each time the rule is encountered.) For example, the rule 2: 1 3 | 3 1 means that to match rule 2, the text being checked must match rule 1 followed by rule 3 or it must match rule 3 followed by rule 1,

Fortunately, there are **no loops in the rules**, so the list of possible matches will be finite. Since rule 1 matches a and rule 3 matches b, rule 2 matches either ab or ba. Therefore, rule 0 matches aab or aba.

The **received messages** (the bottom part of your puzzle input) need to be checked against the rules so you can determine which are valid and which are corrupted.

Your goal is to determine **the number of messages that completely match rule 0**. In the above example, ababbb and abbbab match, but bababa, aaabbb, and aaaabbb do not, producing the answer 2. The whole message must match all of rule 0; there can't be extra unmatched characters in the message. (For example, aaaabbb might appear to match rule 0 above, but it has an extra unmatched b on the end.)

How many messages completely match rule 0?

In [1]:
with open('input-day19.txt', 'r') as fic:
    data = fic.read().rstrip('\n').split('\n\n')
    input1 = data[0].split('\n')
    messages = data[1].split('\n')

In [2]:
rules = {x.split(':')[0]: x.split(':')[1].strip(' ').strip('"') for x in input1}
for k, v in rules.items():
    rules[k] = v.split()

In [3]:
def match_rule(message, rule, rules):
    if (len(rule) == 0):
        return ((len(message) == 0))
    elif (rule[0] in 'abcdefghijklmnopqrstuvwxyz'):
        if (message[0] == rule[0]):
            return match_rule(message[1:], rule[1:], rules)
        else:
            return False
    elif '|' in rule:
        nb = rule.index('|')
        return (match_rule(message, rule[:nb] + rule[(2 * nb + 1):], rules) | 
                match_rule(message, rule[(nb + 1):], rules))
    else:
        return match_rule(message, rules[rule[0]] + rule[1:], rules)

In [4]:
sum([match_rule(message, rules['0'], rules) for message in messages])

113

### --- Part Two ---

As you look over the list of messages, you realize your matching rules aren't quite right. To fix them, completely **replace rules 8**: 42 and **11**: 42 31 with the following:

* **8: 42 | 42 8**
* **11: 42 31 | 42 11 31**

This small change has a big impact: now, the rules do contain loops, and the list of messages they could hypothetically match is infinite. You'll need to determine how these changes affect which messages are valid.

Fortunately, many of the rules are unaffected by this change; it might help to start by looking at which rules always match the same set of values and how those rules (especially rules 42 and 31) are used by the new versions of rules 8 and 11.

(Remember, you only need to handle the rules you have; building a solution that could handle any hypothetical combination of rules would be significantly more difficult.)

After updating rules 8 and 11, how many messages completely match rule 0?

In [5]:
with open('input-day19.txt', 'r') as fic:
    data = fic.read().rstrip('\n').split('\n\n')
    input1 = data[0].split('\n')
    messages = data[1].split('\n')

In [6]:
rules = {x.split(':')[0]: x.split(':')[1].strip(' ').strip('"') for x in input1}
for k, v in rules.items():
    rules[k] = v.split()

In [7]:
rules['8'] = '42 | 42 8'.split()
rules['11'] = '42 31 | 42 11 31'.split()

In [8]:
for k, v in rules.items():
    if '|' in v:
        cut = v.index('|')
        rules[k] = {'type': 'or',
                    'value': [{'type': 'and', 'value': v[:cut]}, {'type': 'and', 'value': v[(cut + 1):]}]}
    else:
        rules[k] = {'type': 'and', 'value': v}

In [9]:
def match_rule(message, rule, rules):
    value = rule['value']
    rule_type = rule['type']
    if (rule_type == 'or'):
        return match_rule(message, value[0], rules) | match_rule(message, value[1], rules)
    elif (rule_type == 'and'):
        if (len(value) == 0):
            return ((len(message) == 0))
        elif (len(message) == 0):
            return False
        elif (value[0] in 'abcdefghijklmnopqrstuvwxyz'):
            if (message[0] == value[0]):
                return match_rule(message[1:], {'type': 'and', 'value': value[1:]}, rules)
            else:
                return False
        else:
            if rules[value[0]]['type'] == 'and':
                return match_rule(message,
                                  {'type': 'and', 'value': rules[value[0]]['value'] + value[1:]},
                                  rules)
            else:
                return ((match_rule(message, 
                                    {'type': 'and', 'value': rules[value[0]]['value'][0]['value'] + value[1:]},
                                    rules)) | 
                        (match_rule(message, 
                                    {'type': 'and', 'value': rules[value[0]]['value'][1]['value'] + value[1:]},
                                    rules)))

In [10]:
sum([match_rule(message, rules['0'], rules) for message in messages])

253

## --- Day 20: Jurassic Jigsaw ---

The high-speed train leaves the forest and quickly carries you south. You can even see a desert in the distance! Since you have some spare time, you might as well see if there was anything interesting in the image the Mythical Information Bureau satellite captured.

After decoding the satellite messages, you discover that the data actually contains many small images created by the satellite's camera array. The camera array consists of many cameras; rather than produce a single square image, they produce many smaller square image tiles that need to be reassembled back into a single image.

Each camera in the camera array returns a single monochrome image tile with a random **unique ID number**. The tiles arrived in a random order.

Worse yet, the camera array appears to be malfunctioning: each image tile has been **rotated and flipped** to a random orientation. Your first task is to reassemble the original image by orienting the tiles so they fit together.

To show how the tiles should be reassembled, each tile's image data includes **a border that should line up exactly with its adjacent tiles**. All tiles have this border, and the border lines up exactly when the tiles are both oriented correctly. Tiles at the edge of the image also have this border, but the outermost edges won't line up with any other tiles.

Assemble the tiles into an image. What do you get if you **multiply together the IDs of the four corner tiles**?

### --- Part Two ---
Now, you're ready to check the image for sea monsters.

The borders of each tile are not part of the actual image; **start by removing them**.
Remove the gaps to form the actual image.
Now, you're ready to search for sea monsters! Because your image is monochrome, a sea monster will look like this:

In [2]:
print('                  # ')
print('#    ##    ##    ###')
print(' #  #  #  #  #  #   ')

                  # 
#    ##    ##    ###
 #  #  #  #  #  #   


When looking for this pattern in the image, the spaces can be anything; **only the # need to match**. Also, you **might need to rotate or flip** your image before it's oriented correctly to find sea monsters. 

Determine how rough the waters are in the sea monsters' habitat by **counting the number of # that are not part of a sea monster**. 

How many # are not part of a sea monster?

## --- Day 21: Allergen Assessment ---

You reach the train's last stop and the closest you can get to your vacation island without getting wet. There aren't even any boats here, but nothing can stop you now: you build a raft. You just need a few days' worth of food for your journey.

You don't speak the local language, so you can't read any ingredients lists. However, sometimes, allergens are listed in a language you do understand. You should be able to use this information to determine which ingredient contains which allergen and work out which foods are safe to take with you on your trip.

You start by compiling a list of foods (your puzzle input), one food per line. Each line includes that food's ingredients list followed by some or all of the allergens the food contains.

Each allergen is found in **exactly one ingredient**. **Each ingredient contains zero or one allergen**. Allergens **aren't always marked**; when they're listed (as in (contains nuts, shellfish) after an ingredients list), the ingredient that contains each listed allergen will be somewhere in the corresponding ingredients list. However, even if an allergen isn't listed, the ingredient that contains that allergen could still be present: maybe they forgot to label it, or maybe it was labeled in a language you don't know.

The first step is to determine **which ingredients can't possibly contain any of the allergens** in any food in your list. In the above example, none of the ingredients kfcds, nhms, sbzzf, or trh can contain an allergen. Counting the number of times any of these ingredients appear in any ingredients list produces 5: they all appear once each except sbzzf, which appears twice.

Determine which ingredients cannot possibly contain any of the allergens in your list. How many times do any of those ingredients appear?

In [1]:
with open('input-day21.txt', 'r') as fic:
    tempo = fic.read().strip('\n').split('\n')

In [2]:
import pandas as pd
all_products = []
parsed = []
for product in tempo:
    ingredients, allergens = product.strip(')').split('(')
    ingredients0 = [f'I_{x}' for x in ingredients.strip(' ').split(' ')]
    allergens0 = [f'A_{x}' for x in allergens[8:].strip(' ').split(', ')]
    parsed += [ingredients0 + allergens0]
    all_products += ingredients0
    all_products += allergens0

all_products = list(set(all_products))
all_products.sort()    

products = pd.DataFrame(columns= all_products)

In [3]:
for product in parsed:
    products = products.append(pd.DataFrame({x: [1] for x in product}))
products.fillna(0, inplace=True)

In [4]:
guess_allergens = {}
for allergen in [col for col in products.columns if col.startswith('A_')]:
    ok = products.loc[products[allergen] == 1, [col for col in products if col.startswith('I_')]].min()
    guess_allergens[allergen] = list(ok.loc[ok == 1].index)

In [5]:
sure_allergens = dict.fromkeys(guess_allergens.keys(), None)
while None in sure_allergens.values():
    for allergen in guess_allergens:
        if len(guess_allergens[allergen]) == 1:
            guessed_product = guess_allergens[allergen][0]
            sure_allergens[allergen] = guessed_product
            for k in guess_allergens:
                guess_allergens[k] = [x for x in guess_allergens[k] if x != guessed_product]

In [6]:
products.loc[
    :, 
    [col for col in products.columns if ((col.startswith('I_')) and (col not in sure_allergens.values()))]
].sum().sum()

2282

### --- Part Two ---
Now that you've isolated the inert ingredients, you should have enough information to figure out which ingredient contains which allergen.

Arrange the ingredients alphabetically by their allergen and separate them by commas to produce your canonical dangerous ingredient list. (There should not be any spaces in your canonical dangerous ingredient list.) In the above example, this would be mxmxvkd,sqjhc,fvjkl.

Time to stock your raft with supplies. What is your canonical dangerous ingredient list?

In [7]:
canonical = pd.DataFrame()
canonical['allergen'] = sure_allergens.keys()
canonical['ingredient'] = sure_allergens.values()

In [8]:
','.join([x[2:] for x in canonical.sort_values(by='allergen')['ingredient']])

'vrzkz,zjsh,hphcb,mbdksj,vzzxl,ctmzsr,rkzqs,zmhnj'

## --- Day 22: Crab Combat ---
It only takes a few hours of sailing the ocean on a raft for boredom to sink in. Fortunately, you brought a small deck of space cards! You'd like to play a game of Combat, and there's even an opponent available: a small crab that climbed aboard your raft before you left.

Fortunately, it doesn't take long to teach the crab the rules.

Before the game starts, split the cards so each player has their own deck (your puzzle input). Then, the game consists of a series of rounds: both players **draw their top card**, and the player with the **higher-valued card wins** the round. The winner keeps both cards, **placing them on the bottom** of their own deck so that the **winner's card is above** the other card. If this causes a player to **have all of the cards**, they win, and the game ends.

#### == Post-game results ==

Once the game ends, you can calculate the winning player's score. The **bottom card** in their deck is worth the value of the card **multiplied by 1**, the second-from-the-bottom card is worth the value of the card multiplied by 2, and **so on**. With 10 cards, the **top card** is worth the value on the card **multiplied by 10**. 

Play the small crab in a game of Combat using the two decks you just dealt. What is the winning player's score?

In [1]:
with open('input-day22.txt', 'r') as fic:
    tempo = fic.read().strip('\n').split('\n\n')

In [2]:
players = [[int(y) for y in x.split(':')[1].strip('\n').split('\n')] for x in tempo]

In [3]:
def crab_combat(players):
    while (min([len(x) for x in players]) > 0):
        if players[0][0] > players[1][0]:
            players = [players[0][1:] + [players[0][0], players[1][0]], players[1][1:]]
        else:
            players = [players[0][1:], players[1][1:] + [players[1][0], players[0][0]]]
    return players

In [4]:
def score(deck):
    return sum([deck[-n] * n for n in range(1, len(deck) + 1)])

In [5]:
final_deck = crab_combat(players)

In [6]:
score(final_deck[0]) + score(final_deck[1])

32033

### --- Part Two ---
You lost to the small crab! Fortunately, crabs aren't very good at recursion. To defend your honor as a Raft Captain, you challenge the small crab to a game of Recursive Combat.

Recursive Combat still starts by splitting the cards into two decks (you offer to play with the same starting decks as before - it's only fair). Then, the game consists of a series of rounds with a few changes:

* Before either player deals a card, if there was **a previous round in this game that had exactly the same cards in the same order** in the same players' decks, the game instantly ends in a win for **player 1**. Previous rounds from other games are not considered. (This prevents infinite games of Recursive Combat, which everyone agrees is a bad idea.)
* Otherwise, this round's cards must be in a new configuration; the players begin the round by each drawing the top card of their deck as normal.
* If **both players have at least as many cards remaining** in their deck as **the value of the card they just drew**, the winner of the round is determined by playing a new game of Recursive Combat (see below).
* Otherwise, at least one player must not have enough cards left in their deck to recurse; the winner of the round is the player with the higher-value card.
* As in regular Combat, the winner of the round (even if they won the round by winning a sub-game) takes the two cards dealt at the beginning of the round and places them on the bottom of their own deck (again so that the winner's card is above the other card). Note that the winner's card might be the lower-valued of the two cards if they won the round due to winning a sub-game. If collecting cards by winning the round causes a player to have all of the cards, they win, and the game ends.

During a round of Recursive Combat, if both players have at least as many cards in their own decks as the number on the card they just dealt, the winner of the round is determined by recursing into a sub-game of Recursive Combat. (For example, if player 1 draws the 3 card, and player 2 draws the 7 card, this would occur if player 1 has at least 3 cards left and player 2 has at least 7 cards left, not counting the 3 and 7 cards that were drawn.)

To play a sub-game of Recursive Combat, each player creates a new deck by making a copy of the next cards in their deck (the quantity of cards copied is equal to the number on the card they drew to trigger the sub-game). During this sub-game, the game that triggered it is on hold and completely unaffected; no cards are removed from players' decks to form the sub-game. (For example, if player 1 drew the 3 card, their deck in the sub-game would be copies of the next three cards in their deck.)

After the game, the winning player's score is calculated from the cards they have in their original deck using the same rules as regular Combat.

Defend your honor as Raft Captain by playing the small crab in a game of Recursive Combat using the same two decks as before. What is the winning player's score?

In [7]:
players = [[9, 2, 6, 3, 1], [5, 8, 4, 7, 10]]

In [8]:
def score(deck):
    return sum([deck[-n] * n for n in range(1, len(deck) + 1)])

In [9]:
def move_cards(players, winner):
    # return a new deck for the 2 players
    if winner == 0:
        return [players[0][1:] + [players[0][0], players[1][0]], players[1][1:]]
    else:
        return [players[0][1:], players[1][1:] + [players[1][0], players[0][0]]]

In [10]:
from copy import deepcopy

In [11]:
def crab_combat(players, previous_decks=[]):
    while (min([len(x) for x in players]) > 0):
        if players in previous_decks:
            #     === Rule: short end if game already played  ===
            return [players[0], []]
        elif (len(players[0]) > players[0][0]) & (len(players[1]) > players[1][0]):
            #     ===           Recursive game entry          ===
            p1 = deepcopy(players[0][1:(players[0][0] + 1)])
            p2 = deepcopy(players[1][1:(players[1][0] + 1)])
            result = crab_combat([p1, p2], [])
            previous_decks += [deepcopy(players)]
            # Use recursive game result to move decks
            if len(result[0]) == 0:
                players = move_cards(players, 1)
            elif len(result[1]) == 0:
                players = move_cards(players, 0)
            else:
                raise(Exception("Rien ne va plus !!!"))
        elif players[0][0] > players[1][0]:
            previous_decks += [deepcopy(players)]
            players = move_cards(players, 0)
        else:
            previous_decks += [deepcopy(players)]
            players = move_cards(players, 1)
        
    return players

In [12]:
final_deck = crab_combat(players)

In [13]:
score(final_deck[0]) + score(final_deck[1])

291

## --- Day 23: Crab Cups ---

The small crab challenges you to a game! The crab is going to mix up some cups, and you have to predict where they'll end up.

The cups will be **arranged in a circle** and labeled clockwise.

Before the crab starts, it will designate the first cup in your list as the current cup. The crab is then going to do 100 moves.

Each move, the crab does the following actions:

* The crab **picks up the three cups that are immediately clockwise** of the current cup. They are removed from the circle; cup spacing is adjusted as necessary to maintain the circle.
* The crab **selects a destination cup**: the cup with a label equal to the current cup's label minus one. If this would select one of the cups that was just picked up, the crab will keep subtracting one until it finds a cup that wasn't just picked up. If at any point in this process the value goes below the lowest value on any cup's label, it wraps around to the highest value on any cup's label instead.
* The crab **places the cups it just picked up** so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
* The crab **selects a new current cup**: the cup which is immediately clockwise of the current cup.

After the crab is done, what order will the cups be in? **Starting after the cup labeled 1**, collect the other cups' labels clockwise into a single string with no extra characters; each number except 1 should appear exactly once.

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

Your puzzle input is 962713854.

In [1]:
puzzle_input = '962713854'

In [2]:
cups = [int(x) for x in puzzle_input]
cups

[9, 6, 2, 7, 1, 3, 8, 5, 4]

In [3]:
position = 0

for n in range(100):
    current = cups[position]
    places_picked_up = [x%9 for x in range(position+1, position+4)]
    picked_up = [cups[n] for n in places_picked_up]
    tempo_cups = [cups[n] for n in range(9) if n not in places_picked_up]
    
    destination = None
    value = current - 1
    while destination is None:
        if value == 0:
            value = 9
        if value in tempo_cups:
            destination = value
        value -= 1
        if value == 0:
            value = 9
    insert_place = tempo_cups.index(destination) + 1
    cups = tempo_cups[:insert_place] + picked_up + tempo_cups[insert_place:]

    position = (cups.index(current) + 1)%9

In [4]:
index1 = cups.index(1)
''.join([str(x) for x in cups[(index1 + 1):] + cups[:index1]])

'65432978'

### --- Part Two ---
Due to what you can only assume is a mistranslation (you're not exactly fluent in Crab), you are quite surprised when the crab starts arranging many cups in a circle on your raft - **one million (1000000) in total**.

Your labeling is still correct for the first few cups; after that, the remaining cups are just numbered in an increasing fashion starting from the number after the highest number in your list and proceeding one by one until one million is reached. (For example, if your labeling were 54321, the cups would be numbered 5, 4, 3, 2, 1, and then start counting up from 6 until one million is reached.) In this way, every number from one through one million is used exactly once.

After discovering where you made the mistake in translating Crab Numbers, you realize the small crab isn't going to do merely 100 moves; the crab is going to do **ten million (10000000) moves**!

The crab is going to hide your stars - one each - under the **two cups that will end up immediately clockwise of cup 1**. You can have them if you predict what the labels on those cups will be when the crab is finished.

Determine which two cups will end up immediately clockwise of cup 1. What do you get if you multiply their labels together?

In [5]:
puzzle_input = '962713854'
ncups = 1000000
nrounds = 10000000

In [6]:
def moins1(nb, nbmax):
    return (nb - 2)%nbmax + 1

In [7]:
def suivant(nb, nbmax):
    return (nb%nbmax + 1)

In [8]:
def list2dict(list_cups):
    nmax = len(list_cups)
    dict_cups = {
        list_cups[n]: [list_cups[(n + 1)%nmax], moins1(list_cups[n], nmax)] 
        for n in range(nmax)
    }
    return dict_cups

In [9]:
def droite(nb, cups):
    return cups[nb][0]

In [10]:
def dict2list(dict_cups, from_cup=1, nb_cups=9):
    list_cups = [from_cup]
    for n in range(nb_cups - 1):
        list_cups += [droite(list_cups[-1], dict_cups)]
    return list_cups

In [11]:
def take3cups(current, cups):
    tercet = dict2list(cups, current, 4)[1:]
    cups[current] = [cups[tercet[2]][0], cups[current][1]]
    return (cups, tercet)

In [12]:
def update_previous(cups, forbidden, nmax):
    to_be_changed = [n%nmax + 1 for n in forbidden]
    previous = [moins1(n, nmax) for n in forbidden]
    for n in range(len(to_be_changed)):
        while previous[n] in forbidden:
            previous[n] = moins1(previous[n], nmax)
        cups[to_be_changed[n]] = [cups[to_be_changed[n]][0], previous[n]]
    return cups

In [13]:
import time
start_time = time.time()

cups = list2dict([int(x) for x in puzzle_input]  + [n for n in range(10, ncups + 1)])
print(f'-- move {1}')
print([int(x) for x in puzzle_input])

print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

current = int(puzzle_input[0])
for n in range(nrounds):
    # Prendre les 3 tasses suivantes
    (cups, tercet) = take3cups(current, cups)
    # Mettre à jour ce que signifie "moins 1" une fois les 3 tasses interdites
    cups = update_previous(cups, tercet, ncups)
    # Définir le lieu où replacer les 3 tasses
    left = cups[current][1]
    right = droite(left, cups)
    # Replacer le tercet
    cups[left] = [tercet[0], cups[left][1]]
    cups[tercet[2]] = [right, cups[tercet[2]][1]]
    # Redéfinir tous les précédents, sans interdiction
    for c in tercet:
        c1 = suivant(c, ncups)
        cups[c1] = [cups[c1][0], moins1(c1, ncups)]
    # Définir la nouvelle tasse courante
    current = cups[current][0]
    
    if (n%300000 == 0):
        print(f'-- {n} --')

print("--- %s seconds ---" % (time.time() - start_time))

-- move 1
[9, 6, 2, 7, 1, 3, 8, 5, 4]
--- 0.9709103107452393 seconds ---
-- 0 --
-- 300000 --
-- 600000 --
-- 900000 --
-- 1200000 --
-- 1500000 --
-- 1800000 --
-- 2100000 --
-- 2400000 --
-- 2700000 --
-- 3000000 --
-- 3300000 --
-- 3600000 --
-- 3900000 --
-- 4200000 --
-- 4500000 --
-- 4800000 --
-- 5100000 --
-- 5400000 --
-- 5700000 --
-- 6000000 --
-- 6300000 --
-- 6600000 --
-- 6900000 --
-- 7200000 --
-- 7500000 --
-- 7800000 --
-- 8100000 --
-- 8400000 --
-- 8700000 --
-- 9000000 --
-- 9300000 --
-- 9600000 --
-- 9900000 --
--- 92.52961254119873 seconds ---


In [14]:
cups[1][0] * cups[cups[1][0]][0]

287230227046

## --- Day 24: Lobby Layout ---
Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.

As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the new tile floor.

The tiles are all **hexagonal**; they need to be arranged in a **hex grid** with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.

The tiles are all white on one side and black on the other. They **start with the white side facing up**. The lobby is large enough to fit whatever pattern might need to appear there.

A member of the renovation crew gives you a **list of the tiles that need to be flipped over** (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a **series of steps starting from a reference tile** in the very center of the room. (Every line starts from the same reference tile.)

Because the tiles are hexagonal, every tile has six neighbors: **east, southeast, southwest, west, northwest, and northeast**. These directions are given in your list, respectively, as **e, se, sw, w, nw, and ne**. A tile is identified by a series of these directions with no delimiters; for example, esenee identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.

**Each time a tile is identified, it flips** from white to black or from black to white. Tiles might be flipped more than once. 

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?

In [1]:
with open('input-day24.txt', 'r') as fic:
    tiles = fic.read().strip('\n').split('\n')

In [2]:
def extract_path(tile):
    path = []
    while len(tile) > 0:
        if tile[0] in ['s', 'n']:
            path += [tile[:2]]
            tile = tile[2:]
        else:
            path += [tile[:1]]
            tile = tile[1:]
    return path

In [3]:
def move(position: tuple, direction: str):
    if direction == 'e':
        return (position[0] + 2, position[1])
    elif direction == 'se':
        return (position[0] + 1, position[1] - 1)
    elif direction == 'sw':
        return (position[0] - 1, position[1] - 1)
    elif direction == 'w':
        return (position[0] - 2, position[1])
    elif direction == 'nw':
        return (position[0] - 1, position[1] + 1)
    elif direction == 'ne':
        return (position[0] + 1, position[1] + 1)
    else:
        raise Exception('Direction non found')

In [4]:
def move_to_tile(path: list, position=(0, 0)):
    for p in path:
        position = move(position, p)
    return position

In [5]:
# Pavement = dictionnaire de positions, indiquant la couleur (True = black / False = White)

def turn_tile(position, pavement):
    if position in pavement:
        pavement[position] = not pavement[position]
    else:
        pavement[position] = True
    return pavement

In [6]:
pathes = [extract_path(tile) for tile in tiles]
lobby = {(0, 0): False}
for p in pathes:
    lobby = turn_tile(move_to_tile(p), lobby)

In [7]:
sum(list(lobby.values()))

424

### --- Part Two ---
The tile floor in the lobby is meant to be a living art exhibit. Every day, the tiles are all flipped according to the following rules:

* Any **black tile** with **zero or more than 2 black** tiles immediately adjacent to it is flipped to white.
* Any **white tile** with **exactly 2 black tiles** immediately adjacent to it is flipped to black.

Here, tiles immediately adjacent means the six tiles directly touching the tile in question.

The rules are applied **simultaneously** to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.

How many tiles will be black after 100 days?

In [8]:
def neighbours(tile: tuple):
    return [
        (tile[0] + 2, tile[1]), (tile[0] + 1, tile[1] - 1), (tile[0] - 1, tile[1] - 1), 
        (tile[0] - 2, tile[1]), (tile[0] - 1, tile[1] + 1), (tile[0] + 1, tile[1] + 1)]

In [9]:
def nb_blacks(tile, pavement):
    nb = 0
    for neighbour in neighbours(tile):
        if neighbour in pavement:
            nb += pavement[neighbour]
    return nb

In [10]:
def enlarge_your_pavement(pavement):
    enlarged = pavement.copy()
    for tile in pavement:
        for adjacent in neighbours(tile):
            if adjacent not in enlarged:
                enlarged[adjacent] = False
    return enlarged

In [11]:
def flip_all(pavement):
    enlarged = enlarge_your_pavement(pavement)
    new_pavement = enlarged.copy()
    counts = {tile: nb_blacks(tile, enlarged) for tile in enlarged}
    for tile in enlarged:
        if enlarged[tile]:
            if (counts[tile] == 0) | (counts[tile] > 2):
                new_pavement[tile] = False
        else:
            if (counts[tile] == 2):
                new_pavement[tile] = True
    return new_pavement

In [12]:
previous_lobby = lobby.copy()

for n in range(100):
    next_lobby = flip_all(previous_lobby)
    previous_lobby = next_lobby

In [13]:
sum(list(next_lobby.values()))

3737

## --- Day 25: Combo Breaker ---

Unfortunately for the door, you know a thing or two about cryptographic handshakes.

The handshake used by the card and the door involves an operation that **transforms a subject number**. To transform a subject number, start with the **value 1**. Then, a number of times called the **loop size**, perform the following steps:

* Set the value to **itself multiplied by the subject number**.
* Set the value to **the remainder after dividing the value by 20201227**.

The card always uses a specific, secret loop size when it transforms a subject number. The door always uses a different, secret loop size.

The cryptographic handshake works like this:

* The **card** transforms the subject number of **7** according to the **card's secret loop size**. The result is called the **card's public key**.
* The **door** transforms the subject number of **7** according to the **door's secret loop size**. The result is called the **door's public key**.
* The card and door use the wireless RFID signal to **transmit the two public keys (your puzzle input)** to the other device. Now, the card has the door's public key, and the door has the card's public key. Because you can eavesdrop on the signal, you have both public keys, but neither device's loop size.
* The **card** transforms the **subject number of the door's public key** according to the **card's loop size**. The result is the encryption key.
* The **door** transforms the **subject number of the card's public key** according to the **door's loop size**. The result is the same encryption key as the card calculated.

If you can use the two public keys to determine each device's loop size, you will have enough information to calculate the secret encryption key that the card and door use to communicate; this would let you send the unlock command directly to the door!

What encryption key is the handshake trying to establish?

In [1]:
def transform_subject(subject, loops_size):
    value = 1
    for n in range(loops_size):
        value *= subject
        value = value % 20201227
    return value

In [2]:
card_public_key = 10604480
door_public_key = 4126658

In [3]:
def find_loop_size(subject, public_key):
    value = 1
    loop_size = 0
    while value != public_key:
        loop_size += 1
        value *= subject
        value = value % 20201227
    return loop_size

In [4]:
card_loop_size = find_loop_size(7, card_public_key)
card_loop_size

1568743

In [5]:
door_loop_size = find_loop_size(7, door_public_key)
door_loop_size

9709101

In [6]:
transform_subject(card_public_key, door_loop_size)

4968512

In [7]:
transform_subject(door_public_key, card_loop_size)

4968512